## 정보보호와 시스템보안
### Team Project #1

### 팀 정보
- 팀 이름 : 
- 팀 번호 :10
- 팀원 목록 : 이한정 (20191650), 김지홍 (20191572)

---

### 프로젝트 TODO
1. Data Preprocessing
2. Data Scaling
3. IP Voting

### 프로젝트 데이터 셋
- 국민대학교 네트워크 플로우 데이터셋(5시간)   

### 참고자료
- 네트워크 플로우([Network Flow](https://checkmk.com/guides/network-flow-monitoring)):   
한번의 세션에서 출발지(Source IP)와 도착지(Destination IP) 사이에 오가는 패킷에 대한 통계적 데이터
### 주의사항
- 기존에 있는 파일과 폴더의 구조를 변경할 수 없음

### 제출
- `<팀 번호>_Project1.ipynb`파일을 팀장이 제출   
팀 번호 : 구글 스프레드 시트 행 인덱스

In [168]:
%cd "./Security Project1"

[Errno 2] No such file or directory: './Security Project1'
/tf/Hanjeong/Security Project1


In [169]:
from Utils import open_pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

### train, valid 데이터 셋 불러오기
- 가상대학을 통해 다운받은 플로우 데이터를 불러온다.

In [170]:
train_path = r"./train.csv"
valid_path = r"./valid.csv"
test_path = r"./project2_test.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)
test_df = pd.read_csv(test_path)

In [171]:
# 학습, 검증 플로우 데이터 개수 확인
print(len(train_df), len(valid_df), len(test_df))

977270 325560 325963


In [172]:
# 주어진 플로우 컬럼 종류 확인
train_df.keys()

Index(['Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Flags',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max'

In [173]:
# Flags 인코딩

flags_dic = {'C':2, 'E':3, 'U':5, 'A':7, 'P':11, 'R':13, 'S':17, 'F':19}
flags = ['........', '......S.', '...A..S.', '...A.RS.', 'CE.APRS.',
       '...AP...', '...A...F', '...APR..', '...AP..F', '...A..SF',
       '.....RS.', '...A.RSF', '...APRS.', '...A.R..', '...AP.SF',
       'CE....S.', '...A....', 'CE.AP.S.', '...AP.S.', '..U...S.',
       'CE.A.RS.', 'CE.AP.SF', '...A.R.F', '...APR.F', 'CE.A.RSF',
       'CE.A..SF', '.....R..', '.E.A..S.', 'CE.A..S.', '...APRSF', 'C..AP...'] # 테스트에서 밑에 unique 결과에서 처음본 애 있으면 추가해줘야함
encoded_flags = []

for flag in tqdm(flags):
    flag = flag.replace('.','')
    encoded_flag = 1
    for s in flag:
        encoded_flag *= flags_dic[s]
    encoded_flags.append(encoded_flag)
    
for idx, flag in enumerate(flags):
    train_df.loc[train_df['Flags'] == flag, 'Flags'] = encoded_flags[idx]
    valid_df.loc[valid_df['Flags'] == flag, 'Flags'] = encoded_flags[idx]
    test_df.loc[test_df['Flags'] == flag, 'Flags'] = encoded_flags[idx]

100%|██████████| 31/31 [00:00<00:00, 154238.94it/s]


In [174]:
print(train_df['Flags'].unique())
print(valid_df['Flags'].unique())
print(test_df['Flags'].unique())

[17 133 1 7 102 24871 1547 221 91 77 1309 1463 85 119 2261 29393 19019
 17017 13 1729 7854 714 1001 102102 9282 149226 176358 13566 357]
[133 17 1 7 119 102 1547 221 24871 91 77 1309 1463 19019 85 2261 29393
 323323 7854 102102 1001 9282 13 149226 17017 714]
[17 133 1 7 102 24871 1547 221 77 1309 1463 91 85 2261 1001 119 154 29393
 13 7854 102102 9282 19019 149226 13566 1729 17017]


In [175]:
protocols = ['TCP', 'UDP', '0']
encoded_protocols = [1, -1, 0]

for idx, protocol in enumerate(protocols):
    train_df.loc[train_df['Protocol'] == protocol, 'Protocol'] = encoded_protocols[idx]
    valid_df.loc[valid_df['Protocol'] == protocol, 'Protocol'] = encoded_protocols[idx]
    test_df.loc[test_df['Protocol'] == protocol, 'Protocol'] = encoded_protocols[idx]

In [176]:
print(train_df['Protocol'].unique())
print(valid_df['Protocol'].unique())
print(test_df['Protocol'].unique())

[1 0 -1]
[1 0 -1]
[1 0 -1]


In [177]:
# train_df = train_df.sort_values('Timestamp')
# train_df.head(20)

In [178]:
# 3분마다 그룹핑
# time_group = {i : [] for i in range(3, 24*60+4, 3)}
# for row in train_df.itertuples():
#   times = list(map(int, row.Timestamp.split(':')))
#   time = times[0]*60 + times[1] # 분 단위만 생각
#   time_key = (time//3 + 1) * 3
#   time_group[time_key].append(row)

In [179]:
train_df = train_df.rename(columns={'Src Port': 'src_port', 'Dst Port': 'dst_port', 'Src IP': 'src_ip', 'Dst IP': 'dst_ip', 'Flow Duration': 'flow_duration', 'Total Fwd Packet': 'total_fwd_packets', 'Total Bwd packets': 'total_bwd_packets'})
valid_df = valid_df.rename(columns={'Src Port': 'src_port','Dst Port': 'dst_port', 'Src IP': 'src_ip', 'Dst IP': 'dst_ip', 'Flow Duration': 'flow_duration', 'Total Fwd Packet': 'total_fwd_packets', 'Total Bwd packets': 'total_bwd_packets'})
test_df = test_df.rename(columns={'Src Port': 'src_port','Dst Port': 'dst_port', 'Src IP': 'src_ip', 'Dst IP': 'dst_ip', 'Flow Duration': 'flow_duration', 'Total Fwd Packet': 'total_fwd_packets', 'Total Bwd packets': 'total_bwd_packets'})

In [180]:
train_df = train_df.astype({'src_port': 'float', 'dst_port': 'float'})
valid_df = valid_df.astype({'src_port': 'float', 'dst_port': 'float'})
test_df = test_df.astype({'src_port': 'float', 'dst_port': 'float'})

In [181]:
# src_port_count = dict(train_df['src_port'].value_counts())
# dst_port_count = dict(train_df['dst_port'].value_counts())
# num_src_port = float(train_df['src_port'].count())
# num_dst_port = float(train_df['dst_port'].count())

In [182]:
# port CDF 인코딩 테이블 만들기
# for key, value in src_port_count.items():
#   src_port_count[key] = value/num_src_port

# for key, value in dst_port_count.items():
#   dst_port_count[key] = value/num_dst_port

In [183]:
# sorted_src_port_table = dict(sorted(src_port_count.items(), key=lambda x: x[1]))
# prev_value = list(sorted_src_port_table.values())[0]
# prev_idx = 0
# cum_sum = 0
# cum_key = []

# for key, value in sorted_src_port_table.items():
#   if value != prev_value:
#     for k in cum_key:
#         sorted_src_port_table[k] = cum_sum
#     cum_key = [key]
#     cum_sum += value
#     prev_value = value
#   else:
#     cum_key.append(key)
#     cum_sum += value
#     prev_value = value
    
# if cum_key:
#     for k in cum_key:
#         sorted_src_port_table[k] = cum_sum
# sorted_src_port_table

In [184]:
# sorted_dst_port_table = dict(sorted(dst_port_count.items(), key=lambda x: x[1]))
# prev_value = list(sorted_dst_port_table.values())[0]
# prev_idx = 0
# cum_sum = 0
# cum_key = []

# for key, value in sorted_dst_port_table.items():
#   if value != prev_value:
#     for k in cum_key:
#         sorted_dst_port_table[k] = cum_sum
#     cum_key = [key]
#     cum_sum += value
#     prev_value = value
#   else:
#     cum_key.append(key)
#     cum_sum += value
#     prev_value = value
    
# if cum_key:
#     for k in cum_key:
#         sorted_dst_port_table[k] = cum_sum
# sorted_dst_port_table.values()

In [185]:
# for key, value in sorted_src_port_table.items():
#     train_df.loc[train_df['src_port'] == key, 'src_port'] = value
# for key, value in sorted_dst_port_table.items():
#     train_df.loc[train_df['dst_port'] == key, 'dst_port'] = value
# for key, value in sorted_src_port_table.items():
#     valid_df.loc[valid_df['src_port'] == key, 'src_port'] = value
# for key, value in sorted_dst_port_table.items():
#     valid_df.loc[valid_df['dst_port'] == key, 'dst_port'] = value
# # train_df['src_port'].replace(sorted_src_port_table, inplace=True)
# # train_df['dst_port'].replace(sorted_dst_port_table, inplace=True)
# # valid_df['src_port'].replace(sorted_src_port_table, inplace=True)
# # valid_df['dst_port'].replace(sorted_dst_port_table, inplace=True)

# valid_df.loc[valid_df['src_port'] >= 2, 'src_port'] = 0 # train에 등장 안해서 테이블에 없는애들 0으로 변경
# valid_df.loc[valid_df['dst_port'] >= 2, 'dst_port'] = 0

In [186]:
train_df.head(5)

,src_ip,src_port,dst_ip,dst_port,Protocol,Flags,Timestamp,flow_duration,total_fwd_packets,total_bwd_packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,110.15.155.169,13850.0,97.2.159.1,53.0,1,17,03:52:00,3119967,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,110.15.155.169,43842.0,97.2.159.1,53.0,1,17,03:37:20,2991830,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,110.15.155.171,25918.0,97.2.159.1,53.0,1,17,03:10:28,3004658,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
3,238.61.97.110,4445.0,97.2.159.1,53.0,1,133,00:42:24,4402645,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,110.15.155.171,28467.0,97.2.159.1,53.0,1,17,02:45:28,3001784,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


In [187]:
valid_df.head(5)

,src_ip,src_port,dst_ip,dst_port,Protocol,Flags,Timestamp,flow_duration,total_fwd_packets,total_bwd_packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,238.61.97.10,61650.0,97.2.159.1,53.0,1,133,00:47:49,4454322,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,110.15.155.171,35929.0,97.2.159.1,53.0,1,17,04:05:32,2988477,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,238.61.97.10,64652.0,97.2.159.1,53.0,1,133,02:07:05,5092931,2,1,...,20,0.0,0.0,0.0,0.0,5090263.0,0.0,5090263.0,5090263.0,Benign_IP
3,110.15.155.169,42876.0,97.2.159.1,53.0,1,17,03:19:22,2993254,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,238.61.97.10,62593.0,97.2.159.1,53.0,1,133,00:53:13,4534770,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


In [188]:
test_df.head(5)

,src_ip,src_port,dst_ip,dst_port,Protocol,Flags,Timestamp,flow_duration,total_fwd_packets,total_bwd_packets,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,21.122.91.166,23801.0,30.239.23.122,53.0,1,17,03:33:43,2987074,3,0,...,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21.122.91.166,28413.0,30.239.23.122,53.0,1,17,03:32:49,2977944,3,0,...,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21.122.91.166,49550.0,30.239.23.122,53.0,1,17,04:23:46,2995375,3,0,...,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21.122.91.166,38373.0,30.239.23.122,53.0,1,17,04:28:08,3003635,3,0,...,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21.122.91.165,8805.0,30.239.23.122,53.0,1,17,03:46:39,3006388,3,0,...,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Problem #1 : Data Preprocessing

### 조건
- "Label"컬럼을 사용하여, 악성 IP의 플로우는 1, 정상 IP의 플로우는 0으로 구성된 라벨 리스트 생성   
=> `train_y`, `valid_y`(변수명 유지)
- 예외처리 컬럼 "Src IP", "Dst IP", "Protocol", "Timestamp", "Flags", "Label"을 제외,   
8개 컬럼을 제외한 48컬럼을 사용하여 데이터셋 생성   
- 4개의 `list` 모두 csv 파일의 플로우 순서 유지
- hint : 결과 파일을 분석하여 코드를 구상한다.

### 관련 정보
- [pandas](https://pandas.pydata.org/docs/) library, [csv](https://docs.python.org/ko/3/library/csv.html) module
- 플로우 피처 정보(주어진 ppt 특정 부분으로 명시)

### I/O
- input :
    > - `train.csv`
    > - `valid.csv`
- output :   
    > - `train_y` -> `list`
    > - `valid_y` -> `list`
    > - `train_list` -> `list`
    > - `valid_list` -> `list`

In [189]:
#TODO1
from tqdm import tqdm

train_y = []
valid_y = []
# test_y = []

for row in tqdm(train_df.itertuples()):
  if row.Label == 'Benign_IP':
    train_y.append(0)
  else:
    train_y.append(1)
  
for row in tqdm(valid_df.itertuples()):
  if row.Label == 'Benign_IP':
    valid_y.append(0)
  else:
    valid_y.append(1)

dropped_train_df = train_df.drop(["src_ip", "dst_ip", "Timestamp", "Label"], axis=1)
dropped_valid_df = valid_df.drop(["src_ip", "dst_ip", "Timestamp", "Label"], axis=1)
dropped_test_df = test_df.drop(["src_ip", "dst_ip", "Timestamp"], axis=1)

train_list = dropped_train_df.values.tolist()
valid_list = dropped_valid_df.values.tolist()
test_list = dropped_test_df.values.tolist() # 여기까지 고쳤다!

977270it [00:09, 107110.87it/s]
325560it [00:03, 105336.62it/s]


## Problem #2 : Data Scaling

### 조건
- sklearn의 [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)사용
- scaler는 학습 데이터로만 완성해야한다.

### 관련 정보
[numpy](https://numpy.org/devdocs/) 패키지

## I/O
- input :
    > - `train_list`
    > - `valid_list`
- output :   
    > - `train_X` -> `ndarray(float64)`
    > - `valid_X` -> `ndarray(float64)`

In [190]:
from sklearn.preprocessing import StandardScaler 

#TODO2 - StandardScaler
scaler = StandardScaler()
scaler.fit(train_list)
train_X = scaler.transform(train_list)
valid_X = scaler.transform(valid_list)
test_X = scaler.transform(test_list)
# 이거 스케일러 하고 새로만든애들 뒤에 붙이는거 해야함

In [191]:
# train_flags = list(train_df['Flags'])
# train_protocols = list(train_df['Protocol'])
# train_src_port = list(train_df['src_port'])
# train_dst_port = list(train_df['dst_port'])

# valid_flags = list(valid_df['Flags'])
# valid_protocols = list(valid_df['Protocol'])
# valid_src_port = list(valid_df['src_port'])
# valid_dst_port = list(valid_df['dst_port'])

In [192]:
print(len(train_X[0]))
print(len(valid_X[0]))
print(len(test_X[0]))
print(train_X[0])
print(valid_X[0])
print(test_X[0])

50
50
50
[-1.6844975  -0.67657058  0.4930201  -0.18574797 -0.31395812 -0.12750747
 -0.02856103 -0.02166621 -0.0151682  -0.24680348 -0.20318065 -0.18825144
 -0.24735784 -0.22601937 -0.18489796 -0.09565073 -0.20957786 -0.24133655
  0.08958055 -0.31365071 -0.10250236 -0.2113708  -0.24138494  0.00362778
 -0.09606723 -0.0691519  -0.0416791  -0.0752894  -0.06503527 -0.26116481
 -0.02988316 -0.19511221 -0.27179518 -0.25292709 -0.16415666 -0.30635339
 -0.27281861 -0.28985815  0.45524432 -0.19177579 -0.03798513  1.56933206
 -0.37243794 -0.07917481 -0.33719269 -0.37237868 -0.38433064 -0.30590296
 -0.39985642 -0.31999349]
[ 0.94176802 -0.67657058  0.4930201  -0.17850898 -0.26099856 -0.13044531
 -0.02217504 -0.02166621 -0.0151682  -0.24680348 -0.20318065 -0.18825144
 -0.24735784 -0.22601937 -0.18489796  0.02259165  0.25075077 -0.05099518
 -0.15997367 -0.26078083  0.28167258 -0.4004523  -0.05093694  0.52341498
 -0.09606723 -0.0691519  -0.0416791  -0.0752894  -0.06503527 -0.29242238
 -0.01722434 -0.

In [193]:
# train_X = train_X.tolist()
# valid_X = valid_X.tolist()

# for i in range(len(train_src_port)):
#     train_X[i].extend([train_flags[i], train_protocols[i]])
# # train_flags[i], train_protocols[i], train_src_port[i], train_dst_port[i]]
# for i in range(len(valid_src_port)):
#     valid_X[i].extend([valid_flags[i], valid_protocols[i]])
# valid_flags[i], valid_protocols[i], valid_src_port[i], valid_dst_port[i]   

In [568]:
print(len(train_X[0]))
print(len(valid_X[0]))
print(len(train_X))
real_train_X = np.concatenate((train_X, valid_X))
real_train_y = np.concatenate((train_y, valid_y))
print(len(real_train_X))
print(len(test_X[0]))

50
50
977270
1302830
50


In [567]:
print(len(train_X))

977270


In [569]:
real_train_X = np.array(real_train_X)
real_train_y = np.array(real_train_y)
valid_X = np.array(valid_X)
test_X = np.array(test_X)

[-1.6844975  -0.6765706   0.4930201  -0.18574797 -0.3139581  -0.12750746
 -0.02856103 -0.02166621 -0.0151682  -0.24680348 -0.20318066 -0.18825144
 -0.24735785 -0.22601937 -0.18489796 -0.09565073 -0.20957786 -0.24133654
  0.08958055 -0.3136507  -0.10250236 -0.2113708  -0.24138494  0.00362778
 -0.09606723 -0.0691519  -0.0416791  -0.07528941 -0.06503528 -0.2611648
 -0.02988316 -0.1951122  -0.27179518 -0.2529271  -0.16415666 -0.3063534
 -0.2728186  -0.28985816  0.45524433 -0.19177578 -0.03798513  1.5693321
 -0.37243795 -0.07917481 -0.33719268 -0.37237868 -0.38433063 -0.30590296
 -0.39985642 -0.3199935 ]
[ 0.941768   -0.6765706   0.4930201  -0.17850898 -0.26099858 -0.13044532
 -0.02217504 -0.02166621 -0.0151682  -0.24680348 -0.20318066 -0.18825144
 -0.24735785 -0.22601937 -0.18489796  0.02259165  0.25075078 -0.05099517
 -0.15997368 -0.26078084  0.28167257 -0.40045232 -0.05093694  0.52341497
 -0.09606723 -0.0691519  -0.0416791  -0.07528941 -0.06503528 -0.29242238
 -0.01722434 -0.1951122  -0.

In [570]:
#학습을 위한 데이터 타입 변환
real_train_X = real_train_X.astype(np.float32)
valid_X = valid_X.astype(np.float32)
test_X = test_X.astype(np.float32)

real_train_y = np.array(real_train_y, dtype=np.float32)
valid_y = np.array(valid_y, dtype=np.float32)

In [287]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [630]:
# train
model = RandomForestClassifier().fit(train_X, train_y)

# predict
predict_valid = model.predict(valid_X)
predict_test = model.predict(test_X)

# valid score
model.score(valid_X, valid_y)

0.9920721218822951

In [632]:
predict_valid = np.array(predict_valid, dtype="float32")
predict_test = np.array(predict_test, dtype="float32")

In [635]:
# 성능 평가를 위해 예측 값을 반올림
round_valid_y = valid_y.astype("int")
round_predict = np.round(predict)

성능 평가 지표 관련 자료 : https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/   
*주의사항 : 이후 진행할 Problem #3의 채점을 위해 학습 결과가 보기 예시와 동일하게 나오는지 확인

In [636]:
# 플로우 기준 이상 탐지 성능 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print("confusion_matrix\n", confusion_matrix(round_valid_y, round_predict))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_predict): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_predict): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_predict): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_predict): .5f}")

confusion_matrix
 [[188565   1708]
 [   740 134547]]
accuracy	:  0.99248
precision	:  0.98746
recall		:  0.99453
F-1		:  0.99098


In [637]:
# 정상 외부 IP set
outer_benignIP_set = open_pickle("./outer_benign_IP.pkl")
# 악성 외부 IP set
outer_malIP_set = open_pickle("./outer_mal_IP.pkl")
# test 외부 IP set
outer_IP_set = open_pickle("outer_ip_set.pkl")

In [638]:
len(outer_benignIP_set), len(outer_malIP_set)
outer_benignIP_set

{'10.12.193.148',
 '10.141.191.225',
 '10.17.3.155',
 '10.180.160.249',
 '10.181.13.245',
 '10.19.178.124',
 '10.20.248.4',
 '10.218.221.118',
 '10.230.238.178',
 '10.91.107.198',
 '100.197.131.227',
 '100.6.185.221',
 '104.24.10.38',
 '104.25.56.172',
 '104.37.87.41',
 '104.90.132.137',
 '104.91.220.166',
 '104.92.111.63',
 '104.93.90.94',
 '104.96.178.76',
 '105.193.175.64',
 '105.208.36.41',
 '105.252.125.157',
 '105.64.16.70',
 '107.112.185.89',
 '107.113.51.237',
 '107.122.85.14',
 '107.216.119.206',
 '107.217.112.245',
 '107.218.168.125',
 '107.234.158.93',
 '107.235.138.156',
 '107.236.190.153',
 '107.237.104.140',
 '107.36.45.97',
 '107.37.18.222',
 '107.38.194.63',
 '107.39.222.7',
 '107.49.251.213',
 '107.50.18.233',
 '107.51.122.84',
 '107.53.188.27',
 '107.60.4.59',
 '107.61.252.192',
 '109.228.229.231',
 '109.50.215.247',
 '109.53.163.226',
 '11.190.145.24',
 '11.249.174.171',
 '11.46.30.75',
 '11.47.88.105',
 '110.20.188.221',
 '110.25.104.146',
 '110.254.142.253',
 '110.

In [639]:
len(outer_IP_set)
outer_IP_set

{'28.180.147.191',
 '2.157.126.160',
 '7.229.242.44',
 '58.26.131.187',
 '110.216.26.21',
 '234.8.65.103',
 '219.3.182.60',
 '123.30.187.239',
 '80.94.191.105',
 '119.241.223.236',
 '74.138.15.237',
 '87.168.225.20',
 '109.225.246.4',
 '52.252.222.235',
 '184.90.59.58',
 '165.41.23.99',
 '59.63.164.4',
 '122.91.121.188',
 '175.129.41.214',
 '178.129.250.55',
 '4.44.254.241',
 '30.30.95.120',
 '200.78.128.254',
 '37.46.169.208',
 '123.56.37.76',
 '107.178.152.0',
 '91.208.23.195',
 '200.74.187.239',
 '107.142.27.240',
 '237.206.70.82',
 '49.226.203.201',
 '35.48.119.12',
 '166.192.63.168',
 '171.19.239.132',
 '28.143.9.148',
 '58.131.178.9',
 '254.8.15.70',
 '50.74.237.186',
 '219.60.166.242',
 '109.231.119.56',
 '117.83.47.182',
 '47.46.124.158',
 '206.2.6.188',
 '28.133.101.101',
 '27.68.107.218',
 '177.189.153.54',
 '200.110.32.248',
 '194.201.254.247',
 '36.93.154.230',
 '28.129.197.138',
 '81.255.78.79',
 '126.109.190.60',
 '225.243.252.92',
 '4.225.187.229',
 '116.175.103.164',
 '

In [640]:
# Problem #3을 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [641]:
len(labelIP_dict)

1047

## Problem #3 : IP Voting
### 조건
- 모든 flow는 내부IP <-> 외부IP 통신으로 이루어져있다.
- 외부IP는 악성IP, 정상IP로 나뉜다.
- 각 외부 IP별로 1개 이상의 플로우를 가진다.
- valid.csv의 각 외부IP별로 플로우를 분류한다.
- 각 외부 IP에서 속한 플로우가 1개라도 `theta`이상의 predict값을 가진다면 악성 외부 IP로 예측한다.
- 외부IP를 key, 예측 값을 value로 가지는 dictionary를 반환한다.(`labelIP_dict`과 같은 형태)
- 예측 라벨은 악성 외부 IP를 `1`, 정상 외부 IP를 `0`으로 한다. 
- 주어진 모든 `theta`에 대해 작동할 수 있도록 한다. (*채점시 다른 `theta`적용 후 결과 비교 예정)
- hint : csv파일의 순서와 predict의 순서는 일치한다.

### I/O
- input :
    > - `valid.csv`
    > - `predict`
    > - `labelIP_dict`
    > - `theta`
- output :   
    > - `predictIP_dict` -> `dict`

In [642]:
#TODO3
predictIP_dict = dict()
external_IP =list(labelIP_dict.keys())
external_IP_group = dict()

# grouping
for row in tqdm(valid_df.itertuples()):
  src_ip = row.src_ip
  dst_ip = row.dst_ip

  if src_ip in external_IP:
    if src_ip in external_IP_group:
      external_IP_group[src_ip].append(row)
    else:
      external_IP_group[src_ip] = [row]
  else:
    if dst_ip in external_IP_group:
      external_IP_group[dst_ip].append(row)
    else:
      external_IP_group[dst_ip] = [row]

# if proportion 0.7, F-1 Score: 0.983
proportion = 0.55
for key, values in tqdm(external_IP_group.items()):
  predictIP_dict[key] = 0
  cnt = 0
  for value in values:
    if predict_valid[value.Index] == 1:
      cnt += 1
      if cnt/float(len(values)) >= proportion:
#           print(len(values))
          predictIP_dict[key] = 1
          break
    
    
test_predictIP_dict = dict()
test_external_IP_group = dict()

# grouping
for row in tqdm(test_df.itertuples()):
  src_ip = row.src_ip
  dst_ip = row.dst_ip

  if src_ip in outer_IP_set:
    if src_ip in test_external_IP_group:
      test_external_IP_group[src_ip].append(row)
    else:
      test_external_IP_group[src_ip] = [row]
  else:
    if dst_ip in test_external_IP_group:
      test_external_IP_group[dst_ip].append(row)
    else:
      test_external_IP_group[dst_ip] = [row]
    
test_proportion = 0.55
for key, values in tqdm(test_external_IP_group.items()):
  test_predictIP_dict[key] = 0
  cnt = 0
  for value in values:
    if predict_test[value.Index] == 1:
      cnt += 1
      if cnt/float(len(values)) >= test_proportion:
          test_predictIP_dict[key] = 1
          break

325560it [00:06, 47779.36it/s]
100%|██████████| 1047/1047 [00:00<00:00, 1128.13it/s]
325963it [00:06, 46963.78it/s]
100%|██████████| 1047/1047 [00:00<00:00, 1176.60it/s]


In [646]:
# for i in range(len(list(test_predictIP_dict.values()))):
#     if list(predictIP_dict.values())[i] != list(test_predictIP_dict.values())[i]:
#         print(i)

88
234
243
295
451
463
580
782


In [650]:
# 파일 저장
import csv

with open('10_Project2.csv', 'w') as f:
    w = csv.writer(f)
    for key, value in test_predictIP_dict.items():
        w.writerow([key, value])

In [651]:
import pickle

with open("./answer_predictIP_dict_new.pkl", 'wb') as f:
    pickle.dump(predictIP_dict, f)

In [652]:
# 정답 IP dictionary와 예측 IP dictionary를 비교하여 성능 평가
pred_IP = []
label_IP = []

for k in predictIP_dict.keys():
    pred_IP.append(predictIP_dict[k])
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[581   9]
 [  2 455]]
accuracy	:  0.98949
precision	:  0.98060
recall		:  0.99562
F-1		:  0.98806


In [653]:
for proportion in range(0, 100, 5):
    proportion = float(proportion) / 100
    for key, values in tqdm(external_IP_group.items()):
      predictIP_dict[key] = 0
      cnt = 0
      for value in values:
        if predict_valid[value.Index] == 1:
          cnt += 1
          if cnt/float(len(values)) >= proportion:
              predictIP_dict[key] = 1
              break
    
    pred_IP = []
    label_IP = []

    for k in predictIP_dict.keys():
        pred_IP.append(predictIP_dict[k])
        label_IP.append(labelIP_dict[k])

    print(proportion)
    print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
    print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
    print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
    print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
    print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

100%|██████████| 1047/1047 [00:00<00:00, 3713.15it/s]


0.0
confusion_matrix
 [[504  86]
 [  0 457]]
accuracy	:  0.91786
precision	:  0.84162
recall		:  1.00000
F-1		:  0.91400


100%|██████████| 1047/1047 [00:00<00:00, 1672.71it/s]


0.05
confusion_matrix
 [[544  46]
 [  0 457]]
accuracy	:  0.95606
precision	:  0.90855
recall		:  1.00000
F-1		:  0.95208


100%|██████████| 1047/1047 [00:00<00:00, 1566.36it/s]


...

0.85
confusion_matrix
 [[581   9]
 [ 11 446]]
accuracy	:  0.98090
precision	:  0.98022
recall		:  0.97593
F-1		:  0.97807


100%|██████████| 1047/1047 [00:01<00:00, 946.47it/s]


0.9
confusion_matrix
 [[581   9]
 [ 21 436]]
accuracy	:  0.97135
precision	:  0.97978
recall		:  0.95405
F-1		:  0.96674


100%|██████████| 1047/1047 [00:01<00:00, 943.16it/s]

0.95
confusion_matrix
 [[581   9]
 [ 37 420]]
accuracy	:  0.95606
precision	:  0.97902
recall		:  0.91904
F-1		:  0.94808
